# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# import dependencies, libraries, modules and setup
import matplotlib.pyplot as plt #for scatter plots
import pandas as pd #for df built 
import numpy as np #for random lat/lng coordinates
import requests #to pull data
import gmaps #for map
import json
import os

# import API key #security - configured file that holds API keys
from api_keys import g_key 

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# load csv created in WeatherPyUpdate into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tasiilaq,65.61,-37.64,37.40,93,2,6.93,GL,1594100628
1,1,busselton,-33.65,115.33,60.01,73,99,4.00,AU,1594100628
2,2,inuvik,68.35,-133.72,48.20,76,20,10.29,CA,1594100628
3,3,parian dakula,5.56,120.77,83.37,69,97,4.94,PH,1594100628
4,4,kikwit,-5.04,18.82,72.57,42,32,1.90,CD,1594100628


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# heatmap of humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# narrow down cities that fit criteria, drop any results with null values
narrowed_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) \
                                    & (city_data_df["Wind Speed"] < 10) \
                                    & (city_data_df["Cloudiness"] == 0)].dropna()
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
25,25,chekhov,55.14,37.47,77.00,69,0,6.71,RU,1594100629
73,73,nalut,30.33,10.85,74.25,38,0,9.64,LY,1594100632
240,240,pravdinskiy,56.07,37.85,78.01,57,0,6.71,RU,1594100641
242,242,miracema do tocantins,-9.56,-48.40,70.20,54,0,2.82,BR,1594100641
250,250,mazarron,37.60,-1.31,73.00,59,0,1.01,ES,1594100641
317,317,emerald,-23.53,148.17,77.00,23,0,9.17,AU,1594100645
363,363,telde,27.99,-15.42,72.00,83,0,2.24,ES,1594100648
366,366,marfino,55.70,37.38,78.01,69,0,6.71,RU,1594100648
376,376,fuengirola,36.54,-4.62,73.99,68,0,4.70,ES,1594100649
406,406,mount isa,-20.73,139.50,77.00,14,0,3.36,AU,1594100650


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# create DataFrame called hotel_df to store city, country, lat and lng coordinates and hotel name
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
25,chekhov,RU,55.14,37.47,
73,nalut,LY,30.33,10.85,
240,pravdinskiy,RU,56.07,37.85,
242,miracema do tocantins,BR,-9.56,-48.40,
250,mazarron,ES,37.60,-1.31,
317,emerald,AU,-23.53,148.17,
363,telde,ES,27.99,-15.42,
366,marfino,RU,55.70,37.38,
376,fuengirola,ES,36.54,-4.62,
406,mount isa,AU,-20.73,139.50,


In [7]:
# set dict parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# loop through dict 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url endpoints
    name_address = requests.get(base_url, params=params)
    
    # convert to json format
    name_address = name_address.json()
    print(json.dumps(name_address, indent=4, sort_keys=True))
    
    # list first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Hotel info missing...next Hotel")

hotel_df

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 55.10084329999999,
                    "lng": 37.4527198
                },
                "viewport": {
                    "northeast": {
                        "lat": 55.1022312802915,
                        "lng": 37.45406843029149
                    },
                    "southwest": {
                        "lat": 55.0995333197085,
                        "lng": 37.4513704697085
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png",
            "id": "24c6a10e1d9d189bfe4ef358d823c88bc70a58b2",
            "name": "Cottage Edu v gosti",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 1280,
     

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 56.0411087,
                    "lng": 37.908176
                },
                "viewport": {
                    "northeast": {
                        "lat": 56.0424474802915,
                        "lng": 37.9094866802915
                    },
                    "southwest": {
                        "lat": 56.0397495197085,
                        "lng": 37.9067887197085
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "f9dbbd1a0822f55ce855033b40fe8c9b897d0423",
            "name": "Zelenyy Gorodok",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 1365,
                    "html_a

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -9.563598500000001,
                    "lng": -48.39864679999999
                },
                "viewport": {
                    "northeast": {
                        "lat": -9.562321969708496,
                        "lng": -48.3973321697085
                    },
                    "southwest": {
                        "lat": -9.5650199302915,
                        "lng": -48.4000301302915
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "0bf48a981b9473835087639519b0cecf477633b6",
            "name": "Miami Apart Hotel",
            "photos": [
                {
                    "height": 1080,
                    "html_attributions": [
                        "<a href=\"https

{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAAARlG5mKuiKJZGwPit8L1MNinCPWdrms5PMZfQW2XvbHs7rA3KBNsEAT5o8fh5o4Vra3MCB8yXabohT8xBhVdRn4OoByAvyEMfktss6qZFk1QRwnNvIi_IPxdKez5Mr63IE41o9yEESFV3jFz9ZseCFllA1-QZWEXrkYiOs_lra3u-O9S7sRI5o7YePT4dT7yVn130L2O5MJCglm4J81X5hmdEJFi8DiTomZdRkIkfd486eZ7xYxa9cThRYqcjS1hdQ2-stOTbwwtKONLkhRUL2AOFSDzinLCTk4WIBK_q5Yr4dknxMzlHi2RLOuBsu_IWnBVk5Pc05OK1jEYk_Ck-asTzjpK2pFVvc-z1p6Ee-es3I-MOkAMZn-bHidT4Xp5hIQmpkCZC2ul-hrFFFyndN19RoUvrfF6nqHtxbKg9cmyL727HK_Dio",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 37.5636826,
                    "lng": -1.314254
                },
                "viewport": {
                    "northeast": {
                        "lat": 37.5649628802915,
                        "lng": -1.312870919708498
                    },
                    "southwest": {
                        "lat": 37.5622649197085,


{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAAFCoPX6yyxcEhBPA2e1TePkNH47w0lvYXPwB1Tq10vesWl7F7B20irwouUG5-Pq1I85gTQ3L2G2jG5elANVNI4cTxLfYrQsP_9v_X4eM2tU7XdK0wcJbgl3uD54M84bGtPj2xnu2P5nxCdmlqPaIeoJL4FjS-o1lBWsFCIxN7M4c5Hem6LaQKEABSxcitK8FCgVPJRTa9CXfrfLycmXNZLbXVRlyyQHixUbSN5pCkTiNqfBk5kMWQM0kv0tHx9IOFBy8KCIkzuKw6dGYfkdBuFqe7kr0WYDwTF-LXvOHHgw3djnP3STqyyhwGqzZFwhX8hmpY9TwHXP9T-VJNvI7adrKZrj-f7gd6FptBvU3KrhKNtJ9RV0-tZB9uGpOAtMsyhIQB6rcuc1ffwAvjEFEBAg2ohoUpS1cezcWNNwY52h5uqofzLcZO3E",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -23.5262125,
                    "lng": 148.1602479
                },
                "viewport": {
                    "northeast": {
                        "lat": -23.52495696970849,
                        "lng": 148.1615461802915
                    },
                    "southwest": {
                        "lat": -23.5276549302

...

{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAAOOtmubH_5WDhNWLw0rh9mi1y-IzjNKG9KYExrNPAzs5H8fUOYpqaoh2e4p4qQlvOAwdTXCnPK_NIj19Amid4Nzii1glusuzM9PymHkGGmbVNG5aAQLNiCEurqQtjTJItD0c4COrjmdB9gac1MH0nb8U6MIWS2Oh4gMjkBWi_kaFn_gdt3TbGbtWgUxrylQoF2dEe5ZJXgKDQpeKXHeTqAEnf-8JS2vod_i0c4yaqEbrbpb9KWCqOyukk5avg9GEZ_QMsTW7w4GrROyJkVwuFnFh_Tab-U8Y1NNC8ngZvuBwequiinVKsUTWDoljG96W75U-uK0vhURYzGoXg1safxgS48GTCQiCGJ7Bu2lZeycV2tXAqV5bBeEstXBLeOyMjxIQYP1RiIQjGB6x-9BP4rxo0RoUn302OxlM-hLequwOoimHlpOGLFE",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 36.5416667,
                    "lng": -4.619722200000001
                },
                "viewport": {
                    "northeast": {
                        "lat": 36.5431029802915,
                        "lng": -4.618315119708498
                    },
                    "southwest": {
                        "lat": 36.540405

{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAADDi-xiQvs_9oQ5xSaAnaT6Yl5y9q0EQvycuCH6IqUIfOWoAfnvmUpUt0Xpxg8P9qzcvERYh4vmlaSXMMJ7rd9Q5PAQJRxd6_I6LbQ378QuqF8V7JIZoryJzInyDa_lK-8UgHCQfzK7_sp_Uh1y4BFBSonc6UlE1ujbTt2_AfG3zvO44NmjV1vWoMZ_JCV59gSq50XfeTP81liTeL_giQHi5MMCF2dw7h5Ohj0yfUy1fN7aerObfnZSTMkSFem0p-I3wzDglX6O5SQ-vq3Jcu7uQNS1qMOJP8o9s-2lHs2Cq5gNThftWjrQFPIAg9SwNq0VTVSdEWewmmCgTIUvWOQs7zyvVfEP1_bH0wAGakpu3nsZDNWgbybb_Z18zVw7sbxIQ6JqxwZPMvjvKosH2-N-fkxoUUxJC3p302PIpya430A5r5v4J0AE",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -20.72614009999999,
                    "lng": 139.4928025
                },
                "viewport": {
                    "northeast": {
                        "lat": -20.7248044197085,
                        "lng": 139.4942359302915
                    },
                    "southwest": {
                        "lat": -20.7275

{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAAAaxPedv5-RmDpm0Mj_aEm3vWGhg9aF7gHeqb-Qx5o9bX5b7u7Y3M56xUdaLwY-m1P4GGznZlLOf5NXk4FDfbAFpOWVEwA5baF3BZ9SsA4pBI_WqDqKgqgaxJjxyLnJaiL_0laeOpPR_8R5O8W_GpQWJGibkN___3eeGOKXXJzw6wIA2T28BMEZbMy7IxpgIi-8e8IP4U6BZZAoihtqDPE8Ku8MI-3_Awios8ocY4cKZLeyw2s4qp8zQ_gzusfLe9jTkKIVk9B-z30Ec_1Abhg28tbm7d7_Q2al3xWIhPBNEIgdaM-obH0Op2KJuIidv5pjXVNErLz97SgZ3SwUit7RPraBmgU2h-NF5thOlcvg5AEK4iOpAxZdwW9Pp7TJ8tRIQVJv-mP8mlpylT_n2qVDRVhoUC3psgTVyNbt3I58w315S4aobCmc",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 39.2952558,
                    "lng": 44.50542609999999
                },
                "viewport": {
                    "northeast": {
                        "lat": 39.2965707802915,
                        "lng": 44.50678183029149
                    },
                    "southwest": {
                        "lat": 39.29387281

{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAAIrRYFInJvxqN2hv3_0pR9XjqjM70inYoCrkaCAm5swEP0yi_VYmmfrFLKu-y0J8-TIi_S7a7jmfpt8iA7hvMmQ-d6mNXFDmnCCtsO_3FDrajsRob2Bdz9hDbBh5-ARNHRQmsQeJwv4rE0cg3bP6zBzInN3RfjphpI1WMuCH1VHqKrUs9ZWHosOw4m-nhZIxme9hOxG5boWyroFfDkyEo6WcctEek9jSougL-qOvCttyyDNbssjMoPNB5zKgyalgm5g4oTVWb8GCRCTJKfZ3sj5ObKanFnMMVYSHdW2ybQbfBEtsL_U9YZ6JklmZkBC5FVDem7Ru4fSrQqs3-2AGBipv6gIlonslxGl04QxUd7mFnZn7y94acYcXkyL_QMqqdhIQtZkbmR-_Ai69JgJRYlPJshoUZjdSN6IJVcHnZXe7ajr1gGLdGxk",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -8.272012,
                    "lng": -49.260772
                },
                "viewport": {
                    "northeast": {
                        "lat": -8.2708450197085,
                        "lng": -49.2595345697085
                    },
                    "southwest": {
                        "lat": -8.273542980291504,

,City,Country,Lat,Lng,Hotel Name
25,chekhov,RU,55.14,37.47,Cottage Edu v gosti
73,nalut,LY,30.33,10.85,
240,pravdinskiy,RU,56.07,37.85,Zelenyy Gorodok
242,miracema do tocantins,BR,-9.56,-48.40,Miami Apart Hotel
250,mazarron,ES,37.60,-1.31,Apartamentos Oasis de las Palmeras
317,emerald,AU,-23.53,148.17,The Irish Village
363,telde,ES,27.99,-15.42,Hotel Rural Cortijo San Ignacio Golf
366,marfino,RU,55.70,37.38,Kardiologicheskiy Sanatornyy Tsentr Peredelkino
376,fuengirola,ES,36.54,-4.62,Hotel Las Palmeras
406,mount isa,AU,-20.73,139.50,ibis Styles Mt Isa Verona


In [8]:
#NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


KeyError: 'Hotel Name'

In [ ]:
# add marker layer on top of heat map
marker_layer = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)

# Display Map
fig